In [ ]:
## 2. Configuración del entorno de prueba
 
En esta sección documentamos:
- Especificaciones del host físico.
- Sistema operativo de la máquina virtual.
- Imagen base de Docker utilizada.

In [3]:
import platform
import psutil
print("Entorno de la VM - Ubuntu 24.04 LTS Jammy Jellyfish")
print("Sistema Operativo:", platform.system(), platform.release())
print("Procesador (platform):", platform.processor())
print("CPU núcleos lógicos:", psutil.cpu_count(logical=True))
print("CPU núcleos físicos:", psutil.cpu_count(logical=False))
print(f"RAM total: {psutil.virtual_memory().total / (10243):.2f} GB")
print(f"Disco total /: {psutil.disk_usage('/').total / (10243):.2f} GB")

Entorno de la VM - Ubuntu 24.04 LTS Jammy Jellyfish
Sistema Operativo: Linux 6.8.0-52-generic
Procesador (platform): x86_64
CPU núcleos lógicos: 5
CPU núcleos físicos: 5
RAM total: 592053.35 GB
Disco total /: 2509933.87 GB


In [ ]:
### 2.1 Máquina Virtual (VirtualBox)
 
- **Distribución invitada**: Ubuntu 24.04 LTS  
- **Asignación de recursos**:  
  - CPU: 2 vCPUs  
  - Memoria: 4 GB RAM  
  - Disco: 20 GB (dinámico)  

In [4]:
!cat /etc/os-release
!lscpu | grep "Model name"
!free -h | grep Mem

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
Mem:           5,6Gi       2,6Gi       299Mi       164Mi       2,8Gi       2,6Gi


In [ ]:
### 2.2 Contenedor Docker

- Imagen base: `ubuntu:20.04`  
- Recursos “virtuales” (asignados por defecto): mismos núcleos y RAM que el host.

In [3]:
!docker --version
!docker pull ubuntu:20.04
!docker image inspect ubuntu:20.04 | jq '.[0] | {ID: .Id, Size: .Size}'

Docker version 26.1.3, build 26.1.3-0ubuntu1~22.04.1
20.04: Pulling from library/ubuntu
Digest: sha256:8feb4d8ca5354def3d8fce243717141ce31e2c428701f6682bd2fafe15388214
Status: Image is up to date for ubuntu:20.04
docker.io/library/ubuntu:20.04
{
  "ID": "sha256:b7bab04fd9aa0c771e5720bf0cc7cbf993fd6946645983d9096126e5af45d713",
  "Size": 72813617
}


In [ ]:
### 2.3 Validación de recursos dentro del contenedor Docker
- Para asegurar que el contenedor refleja los recursos asignados correctamente, se realiza un test interactivo:

In [1]:
!docker run --rm --cpus=2 -m 4g ubuntu:20.04 \
  bash -c "echo '--- Información dentro del contenedor ---'; \
           lscpu | grep '^CPU(s):'; \
           free -h | grep Mem; \
           df -h /"

--- Información dentro del contenedor ---
CPU(s):                               5
Mem:          5.6Gi       2.7Gi       258Mi       213Mi       2.7Gi       2.5Gi
Filesystem      Size  Used Avail Use% Mounted on
overlay          24G   18G  5.8G  75% /


In [ ]:
### 2.4 Preparación del volumen de resultados
- Para facilitar la recolección de datos y gráficos, crearemos un volumen compartido entre VM y Docker:

In [2]:
# Crear el directorio de resultados y asignar permisos
!mkdir -p ../results
!chmod -R a+rw ../results
!ls -ld ../results

drwxrwxrwx 2 flez flez 4096 may 15 23:23 ../results
